In [ ]:
# external imports
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from matplotlib import ticker as mticker
from pymatgen.core.structure import Structure

# setup for plots
plt.rcParams["text.usetex"] = True
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")
plt.rc("text", usetex=True)

# so python can find the calc directory
os.chdir("../")

The state-of-the-art (SOTA) algorithm mentioned in the paper is labeled
"npj" instead of "sota" in the code for historical reasons.

In [ ]:
# data structures that contain all relevant information
@dataclass
class txt_data:
    name: str  # material name
    structure: Structure  # pymatgen structure class
    data: list  # output of the .txt file
    gwc: list  # list with all output classes


@dataclass
class cdata:
    name: str  # material name
    structure: Structure  # pymatgen structure class
    kppa: np.array  # k-point density
    kpt_str: list  # k-point string
    nkir: np.array  # number of irreducible k-points
    params: np.ndarray  # gw parameters
    gap: np.array  # direct gap at the gamma point
    ngw: np.ndarray  # total number of gw calculations
    tcpu: np.array  # cpu time
    gwc: list  # list with all output classes
    extra: np.array  # gap extrapolation from the fit algorithm

In [ ]:
# parse my txt files
def output_parser(txt_path, calc_type):
    with open(txt_path, "r") as f:
        s = f.readlines()
    s = s[1:]  # skip the header
    if calc_type == "cs":
        data = np.array(-1 * np.ones([5, 10], dtype=int), dtype=str)
    elif calc_type == "npj":
        data = np.array(-1 * np.ones([5, 11], dtype=int), dtype=str)
    elif calc_type == "ref":
        data = np.array(-1 * np.ones([5, 9], dtype=int), dtype=str)
    for i, l in enumerate(s):
        if not "FAILED" in l:
            data[i] = l.split()
    return data

In [ ]:
"""
Parse the results for the 10 2D materials
"""

# initializations
calc_dir = "calc"
mat_list = os.listdir("input_2d")
mat_list = [m.split(".")[0] for m in mat_list]
res_cs = []
res_npj = []
res_ref = []
conv_thr = 50  # meV

# table in the supplement
si_table = []

# loop over all materials
for mp in mat_list:

    # parse the structure
    with open(f"{calc_dir}/{mp}/structure.pckl", "rb") as f:
        structure, name = pickle.load(f)
    for site in structure.sites:  # fix the site labels ...
        site.label = site.species.elements[0].name

    # adjust the name of the material for the figure titles to get the correct latex formatting
    name = name.replace("2", "$_2$")  # we only have elements with 2 in the name
    
    # change the name of BN to hBN
    if name == "BN":
        name = "hBN"

    # collect the data from the cooridnate search algorithm
    cs_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_kpt"
    if os.path.isfile(f"{cs_path}/{mp}_gw_conv_cs.txt"):
        # data from the output text file
        data = output_parser(f"{cs_path}/{mp}_gw_conv_cs.txt", "cs")

        # collect all classes
        cs_class = []
        for kppa in data[:, 0]:
            cs_class_path = f"{cs_path}/kppa{kppa}_cs/g0w0_cs/class_cs.pckl"
            if os.path.isfile(cs_class_path):
                with open(cs_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            cs_class.append(gw_class)

        # create the dataclass for each k-grid
        res_cs.append(
            cdata(
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                cs_class,  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )

    else:
        print(f"{mp} cs is missing...")

    # temporary list for the si table
    # mp-id, name, cs Nb, cs Gcut, cs gap
    temp_si = [
        name,
        data[-1, 4].astype(int),
        data[-1, 5].astype(int),
        f"{data[-1,6].astype(float):.3f}",
    ]

    # collect the data from the npj fit algorithm
    npj_path = f"{calc_dir}/{mp}/yambo_g0w0_npj_kpt/"
    if os.path.isfile(f"{npj_path}/{mp}_gw_conv_npj.txt"):
        # data from the output text file
        data = output_parser(f"{npj_path}/{mp}_gw_conv_npj.txt", "npj")

        # collect all classes
        npj_class = []
        extra = []
        for kppa in data[:, 0]:
            npj_class_path = f"{npj_path}/kppa{kppa}_npj/g0w0_npj/class_npj.pckl"
            if os.path.isfile(npj_class_path):
                with open(npj_class_path, "rb") as f:
                    gw_class = pickle.load(f)
            else:
                gw_class = 0
            npj_class.append(gw_class)
            if gw_class == 0:
                extra.append(-1)
            else:
                extra.append(gw_class.extra)
        extra = np.array(extra)

        # create the dataclass for each k-grid
        res_npj.append(
            cdata(
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameteres
                data[:, 6].astype(float),  # direct gap
                data[:, 7].astype(int),  # number of gw calculations
                data[:, 8].astype(int) * data[:, 9].astype(int),  # cpu time
                npj_class,  # list with all classes
                extra,  # gap extrapolation from the fit algorithm
            )
        )
    else:
        print(f"{mp} npj is missing...")

    # append the npj gap (if the algorithm converged...)
    if data[-1, 6].astype(float) == -1:
        temp_si.append(f"-")
    else:
        temp_si.append(f"{data[-1,6].astype(float):.3f}")

    # collect the data from the cs reference calculations with high computation parameters
    ref_path = f"{calc_dir}/{mp}/yambo_g0w0_cs_reference/"
    if os.path.isfile(f"{ref_path}/{mp}_gw_conv_cs_ref.txt"):
        # data from the output text file
        data = output_parser(f"{ref_path}/{mp}_gw_conv_cs_ref.txt", "ref")

        # create the dataclass for each k-grid
        res_ref.append(
            cdata(
                name,  # material name
                structure,  # pymatgen structure class
                data[:, 0].astype(int),  # kppa
                data[:, 1],  # k-point string
                data[:, 2].astype(int),  # number of irreducible k-points
                data[:, 3:6].astype(int),  # gw parameters
                data[:, 6].astype(float),  # direct gap
                np.ones(5),  # number of gw calculations
                data[:, 7].astype(int) * data[:, 8].astype(int),  # cpu time
                [],  # list with all classes
                np.zeros(5),  # gap extrapolation from the fit algorithm
            )
        )
    else:
        print(f"{mp} ref is missing...")

    # append the ref gap
    temp_si.append(f"{data[-1,6].astype(float):.3f}")

    # append the si table
    si_table.append(temp_si)

# turn the si table into a numpy array
si_table = np.array(si_table)

# number of finished calculations
print(f"#cs  = {len(res_cs)}")
print(f"#npj = {len(res_npj)}")
print(f"#ref = {len(res_ref)}")

In [ ]:
# create a dataframe and convert it to a latex table
var_names = [
    "$N_b$ [CS]",
    "$G_\\textrm{cut}$~(Ry) [CS]",
    "$\\textrm{E}_\\textrm{gap}^{{\\Gamma}-{\\Gamma}}$~(eV) [CS]",
    "$\\textrm{E}_\\textrm{gap}^{{\\Gamma}-{\\Gamma}}$~(eV) [SOTA]",
    "$\\textrm{E}_\\textrm{gap}^{{\\Gamma}-{\\Gamma}}$~(eV) [R]",
]
latex_table = pd.DataFrame(si_table[:, 1:], columns=var_names, index=si_table[:, 0])
latex_table_style = latex_table.style
print(
    latex_table_style.to_latex(
        column_format="cccccc",
        position="h",
        position_float="centering",
        hrules=True,
        label="mp",
        caption="",
    )
)

In [ ]:
# check whether the gaps in the table are converged with respect to the k-point grid
# for this we check the k-point convergence for the calculations with the large parameters in W (ref)
for r in res_ref:
    if np.abs(r.gap[-1] - r.gap[-2]) * 1000 > conv_thr:
        print(f"{r.name}")  # these materials are highlighted in the table in the SI

In [ ]:
# some statistics
total_gw = 0
total_tcpu = 0
for i in range(len(res_cs)):
    idx_used_cs = res_cs[i].ngw > 0
    idx_used_npj = res_cs[i].ngw > 0
    total_gw += np.sum(res_cs[i].ngw[idx_used_cs])
    total_gw += np.sum(res_npj[i].ngw[idx_used_npj])
    total_gw += np.sum(res_ref[i].ngw)
    total_tcpu += np.sum(res_cs[i].tcpu[idx_used_cs])
    total_tcpu += np.sum(res_npj[i].tcpu[idx_used_npj])
    total_tcpu += np.sum(res_ref[i].tcpu)
print(f"Total number of GW calculation for all plots: {int(total_gw):d}!")
print(
    f"Total CPU time for all plots: {np.round(total_tcpu / 3600 / 24 / 365, 1):.1f} years!"
)
print("This includes the reference calculations!")

In [ ]:
# average gap difference in meV, cpu time and number of gw calculations
# for each k-grid with the same number of irreducible k-points as well
# as the computation time needed for each convergence calculation
num_mat = len(res_cs)
num_kpt = len(res_cs[0].kppa)
gap_diff_cs = []
ngw_cs = []
time_cs = []
gap_diff_npj = []
ngw_npj = []
time_npj = []
for i in range(num_mat):
    for j in range(num_kpt):
        if (res_cs[i].nkir[j] > 0) and (res_npj[i].nkir[j] > 0):
            temp_gap_diff_cs = np.abs(res_cs[i].gap[j] - res_ref[i].gap[j]) * 1000
            if temp_gap_diff_cs > 50:
                print(
                    f"CS Large gap difference:  {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:<5d} gap_diff = {temp_gap_diff_cs:.2f} meV"
                )
            gap_diff_cs.append([res_cs[i].nkir[j], temp_gap_diff_cs])
            if res_cs[i].ngw[j] > 11:
                print(
                    f"CS Large number of GW:    {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:<5d} ngw = {res_cs[i].ngw[j]:d}"
                )
            ngw_cs.append([res_cs[i].nkir[j], res_cs[i].ngw[j]])
            time_cs.append([res_cs[i].nkir[j], res_cs[i].tcpu[j]])
            temp_gap_diff_npj = np.abs(res_npj[i].gap[j] - res_ref[i].gap[j]) * 1000
            if temp_gap_diff_npj > 50:
                print(
                    f"NPJ Large gap difference: {res_cs[i].name:<8s} kppa = {res_npj[i].kppa[j]:<5d} gap_diff = {temp_gap_diff_npj:.2f} meV"
                )
            gap_diff_npj.append([res_npj[i].nkir[j], temp_gap_diff_npj])
            if res_npj[i].ngw[j] > 35:
                print(
                    f"NPJ Large number of GW:   {res_npj[i].name:<8s} kppa = {res_npj[i].kppa[j]:<5d} ngw = {res_npj[i].ngw[j]:d}"
                )
            ngw_npj.append([res_npj[i].nkir[j], res_npj[i].ngw[j]])
            time_npj.append([res_npj[i].nkir[j], res_npj[i].tcpu[j]])
            if res_cs[i].tcpu[j] > res_npj[i].tcpu[j]:
                print(
                    f"CS slower than NPJ:       {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:<5d} speedup = {res_npj[i].tcpu[j]/res_cs[i].tcpu[j]:.2f}"
                )
        else:
            print(f"Skipping: {res_cs[i].name:<8s} kppa = {res_cs[i].kppa[j]:d}")
gap_diff_cs = np.array(gap_diff_cs)
ngw_cs = np.array(ngw_cs)
time_cs = np.array(time_cs)
gap_diff_npj = np.array(gap_diff_npj)
ngw_npj = np.array(ngw_npj)
time_npj = np.array(time_npj)

# we ignore the data for both algorithms for a material at a specific k-point grid
# in our analysis if one of the algorithms failed to converge for this specific point
# (only the npj/sota algorithm failed in same test cases, while the cs always worked)

Compare the accuracy and performance of both algorithms.

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.25, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.25, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.25, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.25, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the gap differences
axs[0, 0].hist(gap_diff_cs[:, 1], bins=50, color="tab:blue")
axs[0, 0].axvline(x=conv_thr, color="k", linestyle="-")
axs[0, 0].axvline(x=np.mean(gap_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(gap_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[0, 1].hist(gap_diff_npj[:, 1], bins=50, color="tab:blue")
axs[0, 1].axvline(x=conv_thr, color="k", linestyle="-")
axs[0, 1].axvline(x=np.mean(gap_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(gap_diff_npj[:, 1]), color="limegreen", linestyle="--")

# plot the number of gw calculations
axs[1, 0].hist(
    ngw_cs[:, 1],
    bins=np.arange(np.min(ngw_cs[:, 1]), np.max(ngw_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(ngw_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(ngw_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    ngw_npj[:, 1],
    bins=np.arange(np.min(ngw_npj[:, 1]), np.max(ngw_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(ngw_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(ngw_npj[:, 1]), color="limegreen", linestyle="--")

# axis labels
axs[0, 0].set_xlabel(
    r"$\vert\Delta_\textrm{R}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (meV)",
    size=fs,
)
axs[0, 1].set_xlabel(
    r"$\vert\Delta_\textrm{R}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (meV)",
    size=fs,
)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$\textrm{N}_\textrm{GW}$", size=fs)
axs[1, 1].set_xlabel(r"$\textrm{N}_\textrm{GW}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].set_xlim([0, 200])
axs[0, 0].xaxis.set_ticks([0, 50, 100, 150, 200])
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].set_xlim([0, 200])
axs[0, 1].xaxis.set_ticks([0, 50, 100, 150, 200])
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].set_xlim([2, 14])
axs[1, 0].xaxis.set_ticks([0, 2, 4, 6, 8, 10, 12, 14])
axs[1, 0].yaxis.set_ticks([0, 5, 10, 15, 20])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].xaxis.set_ticks([0, 2, 4, 6, 8, 10, 12, 14])
axs[1, 1].yaxis.set_ticks([0, 5, 10, 15, 20])
axs[1, 1].set_xlim([2, 14])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.65, wspace=0.35)

# save the figure
fig.savefig("plots/2d/conv_analysis.pdf")

In [ ]:
# some statistics for the gap difference to the reference calculation
print("MEAN")
print(f"CS:   {np.mean(gap_diff_cs[:,1]):.2f} meV")
print(f"SOTA: {np.mean(gap_diff_npj[:,1]):.2f} meV\n")

print("MEDIAN")
print(f"CS:   {np.median(gap_diff_cs[:,1]):.2f} meV")
print(f"SOTA: {np.median(gap_diff_npj[:,1]):.2f} meV\n")

print("RANGE")
print(f"CS:   {np.max(gap_diff_cs[:,1]) - np.min(gap_diff_cs[:,1]):.2f} meV")
print(f"SOTA: {np.max(gap_diff_npj[:,1]) - np.min(gap_diff_npj[:,1]):.2f} meV")

In [ ]:
# some statistics for the number of gw calculations needed
print("MEAN")
print(f"CS:   {np.mean(ngw_cs[:,1]):.1f}")
print(f"SOTA: {np.mean(ngw_npj[:,1]):.1f}\n")

print("MEDIAN")
print(f"CS:   {np.median(ngw_cs[:,1]):.1f}")
print(f"SOTA: {np.median(ngw_npj[:,1]):.1f}\n")

print("RANGE")
print(f"CS:   {np.max(ngw_cs[:,1])-np.min(ngw_cs[:,1]):.1f}")
print(f"SOTA: {np.max(ngw_npj[:,1])-np.min(ngw_npj[:,1]):.1f}\n")

In [ ]:
# calculate the speed up
speed_up = time_npj[:, 1] / time_cs[:, 1]

# figure size (tested)
width = 2.65
height = 2.0

# text size
fs = 10

# figure
fig, ax = plt.subplots(1, 1, figsize=(width, height), facecolor="w", dpi=600)

# plot the relative time differences
ax.hist(
    speed_up,
    bins=np.arange(np.min(speed_up), np.max(speed_up), 0.1) - 0.5,
    color="tab:blue",
)
ax.axvline(x=np.mean(speed_up), color="magenta", linestyle="--")
ax.axvline(x=np.median(speed_up), color="limegreen", linestyle="--")
ax.axvline(x=1, color="k", linestyle="--")

# axis ticks
ax.xaxis.set_tick_params(labelsize=fs)
ax.yaxis.set_tick_params(labelsize=fs)
ax.tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
ax.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)
ax.xaxis.set_ticks([0, 2, 4, 6, 8, 10, 12, 14])
ax.yaxis.set_ticks(ax.get_yticks().astype(int))

# axis limit
ax.set_xlim([0, 14])

# axis labels
ax.set_xlabel(r"$\textrm{T}_\textrm{SOTA}/\textrm{T}_\textrm{CS}$", size=fs)
ax.set_ylabel(r"Occurrences", size=fs)

# fix layout
fig.tight_layout()

# save the figure
fig.savefig("plots/2d/cs_speed_up.pdf")

In [ ]:
# some statistics for speed up
print(f"MEAN:   {np.mean(speed_up):.1f}")
print(f"MEDIAN: {np.median(speed_up):.1f}")
print(f"MAX:    {np.max(speed_up):.1f}")

In [ ]:
# compare the total computation times
print(f"Total CS computation time:   {np.sum(time_cs[:,1]) / 3600:.1f} h")
print(f"Total SOTA computation time: {np.sum(time_npj[:,1]) / 3600:.1f} h")

Check how the W convergence depends on the k-point grid.

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the Gamma-only to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[0, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.name}")
    # for annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[0, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.name}")
    # for annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the gamma convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the gamma convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 0].set_xlim([-4, 4])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 1].set_xlim([-4, 4])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].xaxis.set_ticks([-2, 0, 2, 4])
axs[1, 0].set_xlim([-2, 4])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].xaxis.set_ticks([-2, 0, 2, 4])
axs[1, 1].set_xlim([-2, 4])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/2d/param_indep_w_k_1.pdf")

In [ ]:
# check the percentage of how many times the gamma-only grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the k2 to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[1, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.name}")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[1, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.name}")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the gamma convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the gamma convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 0].set_xlim([-4, 4])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 1].set_xlim([-4, 4])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].xaxis.set_ticks([-2, 0, 2, 4])
axs[1, 0].set_xlim([-2, 4])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].xaxis.set_ticks([-2, 0, 2, 4])
axs[1, 1].set_xlim([-2, 4])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/2d/param_indep_w_k_2.pdf")

In [ ]:
# check the percentage of how many times the k2 grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the k3 to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[2, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.name}")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[2, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.name}")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the gamma convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the gamma convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 0].set_xlim([-4, 4])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 1].set_xlim([-4, 4])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].xaxis.set_ticks([-2, -1, 0, 1, 2])
axs[1, 0].yaxis.set_ticks([0, 2, 4, 6, 8, 10])
axs[1, 0].set_xlim([-2, 2])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].xaxis.set_ticks([-2, -1, 0, 1, 2])
axs[1, 1].yaxis.set_ticks([0, 2, 4, 6, 8, 10])
axs[1, 1].set_xlim([-2, 2])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/2d/param_indep_w_k_3.pdf")

In [ ]:
# check the percentage of how many times the k3 grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

In [ ]:
# step size in our algorithm
delta_cs = [100, 4]
delta_npj = [200, 4]

# calculate the relative parameters change in W from the k4 to final k-point grid
# normalized to the step size of the respective convergence algorithm
w_diff_cs = []
w_diff_npj = []
for r in res_cs:
    if not -1 in r.params[:, 0]:
        temp_w_diff_cs = (r.params[3, 1:] - r.params[-1, 1:]) / delta_cs
        w_diff_cs.append(temp_w_diff_cs)
    else:
        print(f"Skipping: {r.name}")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_cs[0] < 0:
        print(f"CS Negative N_b:    {r.name:<10s} {temp_w_diff_cs[0]}")
    if temp_w_diff_cs[1] < 0:
        print(f"CS Negative G_cut:  {r.name:<10s} {temp_w_diff_cs[1]}")
for r in res_npj:
    if not -1 in r.params[:, 0]:
        temp_w_diff_npj = (r.params[3, 1:] - r.params[-1, 1:]) / delta_npj
        w_diff_npj.append(temp_w_diff_npj)
    else:
        print(f"Skipping: {r.name}")
    # check the materials with a negative C_p, but for the supplement we do not do the annotations
    if temp_w_diff_npj[0] < 0:
        print(f"NPJ Negative N_b:   {r.name:<10s} {temp_w_diff_npj[0]}")
    if temp_w_diff_npj[1] < 0:
        print(f"NPJ Negative G_cut: {r.name:<10s} {temp_w_diff_npj[1]}")
w_diff_cs = np.array(w_diff_cs)
w_diff_npj = np.array(w_diff_npj)

In [ ]:
# figure size (tested)
width = 2 * 2.65
height = 4.5

# text size
fs = 10

# figure
fig, axs = plt.subplots(2, 2, figsize=(width, height), facecolor="w", dpi=600)

# add (a), (b), (c) and (d) to the subplots
axs[0, 0].text(-0.2, 1.15, r"\textbf{a}", transform=axs[0, 0].transAxes, size=11)
axs[0, 1].text(-0.18, 1.15, r"\textbf{b}", transform=axs[0, 1].transAxes, size=11)
axs[1, 0].text(-0.2, 1.15, r"\textbf{c}", transform=axs[1, 0].transAxes, size=11)
axs[1, 1].text(-0.18, 1.15, r"\textbf{d}", transform=axs[1, 1].transAxes, size=11)

# plot the change in the number of bands when comparing the gamma convergence to the last k-grid
axs[0, 0].hist(
    w_diff_cs[:, 0],
    bins=np.arange(np.min(w_diff_cs[:, 0]), np.max(w_diff_cs[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 0].axvline(x=np.mean(w_diff_cs[:, 0]), color="magenta", linestyle="--")
axs[0, 0].axvline(x=np.median(w_diff_cs[:, 0]), color="limegreen", linestyle="--")
axs[0, 1].hist(
    w_diff_npj[:, 0],
    bins=np.arange(np.min(w_diff_npj[:, 0]), np.max(w_diff_npj[:, 0]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[0, 1].axvline(x=np.mean(w_diff_npj[:, 0]), color="magenta", linestyle="--")
axs[0, 1].axvline(x=np.median(w_diff_npj[:, 0]), color="limegreen", linestyle="--")

# plot the change in the cutoff when comparing the gamma convergence to the last k-grid
axs[1, 0].hist(
    w_diff_cs[:, 1],
    bins=np.arange(np.min(w_diff_cs[:, 1]), np.max(w_diff_cs[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 0].axvline(x=np.mean(w_diff_cs[:, 1]), color="magenta", linestyle="--")
axs[1, 0].axvline(x=np.median(w_diff_cs[:, 1]), color="limegreen", linestyle="--")
axs[1, 1].hist(
    w_diff_npj[:, 1],
    bins=np.arange(np.min(w_diff_npj[:, 1]), np.max(w_diff_npj[:, 1]) + 5, 1) - 0.5,
    color="tab:blue",
)
axs[1, 1].axvline(x=np.mean(w_diff_npj[:, 1]), color="magenta", linestyle="--")
axs[1, 1].axvline(x=np.median(w_diff_npj[:, 1]), color="limegreen", linestyle="--")

# titles
axs[0, 0].set_title(r"CS", fontsize=fs + 2, pad=15)
axs[0, 1].set_title(r"SOTA", fontsize=fs + 2, pad=15)

# axis labels
axs[0, 0].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 1].set_xlabel(r"$C^\Gamma_{N_b}$", size=fs)
axs[0, 0].set_ylabel(r"Occurrences", size=fs)
axs[1, 0].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 1].set_xlabel(r"$C^\Gamma_{G_\textrm{\scriptsize cut}}$", size=fs)
axs[1, 0].set_ylabel(r"Occurrences", size=fs)

# axis ticks
axs[0, 0].xaxis.set_tick_params(labelsize=fs)
axs[0, 0].yaxis.set_tick_params(labelsize=fs)
axs[0, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 0].yaxis.set_ticks(axs[0, 0].get_yticks().astype(int))
axs[0, 0].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 0].yaxis.set_ticks([0, 2, 4, 6, 8, 10])
axs[0, 0].set_xlim([-4, 4])
axs[0, 0].set_ylim([0, 10.5])
axs[0, 1].xaxis.set_tick_params(labelsize=fs)
axs[0, 1].yaxis.set_tick_params(labelsize=fs)
axs[0, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[0, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[0, 1].yaxis.set_ticks(axs[0, 1].get_yticks().astype(int))
axs[0, 1].xaxis.set_ticks([-4, -2, 0, 2, 4])
axs[0, 1].set_xlim([-4, 4])
axs[1, 0].xaxis.set_tick_params(labelsize=fs)
axs[1, 0].yaxis.set_tick_params(labelsize=fs)
axs[1, 0].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 0].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 0].xaxis.set_ticks([-2, -1, 0, 1, 2])
axs[1, 0].set_xlim([-2, 2])
axs[1, 1].xaxis.set_tick_params(labelsize=fs)
axs[1, 1].yaxis.set_tick_params(labelsize=fs)
axs[1, 1].tick_params(
    bottom=True,
    top=True,
    left=True,
    right=True,
    which="both",
    width=1,
    length=4,
    direction="in",
)
axs[1, 1].tick_params(
    labelbottom=True, labeltop=False, labelleft=True, labelright=False
)
axs[1, 1].xaxis.set_ticks([-2, -1, 0, 1, 2])
axs[1, 1].yaxis.set_ticks([0, 2, 4, 6, 8, 10])
axs[1, 1].set_xlim([-2, 2])

# fix layout
fig.tight_layout()
fig.subplots_adjust(hspace=0.6, wspace=0.25)

# save the figure
fig.savefig("plots/2d/param_indep_w_k_4.pdf")

In [ ]:
# check the percentage of how many times the k4 grid is > 0,
# i.e. that the convergence surface is more flat on the final k-point grid compared to the gamma-only grid
print("CS: C_p >= 0")
print(
    f"{len(w_diff_cs[(w_diff_cs[:,0] >= 0) & (w_diff_cs[:,1] >= 0), 0]) / len(w_diff_cs[:,0]) * 100:.2f} %\n"
)
print("NPJ: C_p >= 0")
print(
    f"{len(w_diff_npj[(w_diff_npj[:,0] >= 0) & (w_diff_npj[:,1] >= 0), 0]) / len(w_diff_npj[:,0]) * 100:.2f} %\n"
)
# we need to check where C_P >= 0 holds for both parameters at the same time

# some statistics for the parameter stability of W with respect to the k-point grid density
print("CS: MEAN")
print(f"N_b:   {np.mean(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_cs[:,1]):.1f}\n")

print("CS: MEDIAN")
print(f"N_b:   {np.median(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_cs[:,1]):.1f}\n")

print("CS: RANGE")
print(f"N_b:   {np.max(w_diff_cs[:,0]) - np.min(w_diff_cs[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_cs[:,1]) - np.min(w_diff_cs[:,1]):.1f}\n")

print("NPJ: MEAN")
print(f"N_b:   {np.mean(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.mean(w_diff_npj[:,1]):.1f}\n")

print("NPJ: MEDIAN")
print(f"N_b:   {np.median(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.median(w_diff_npj[:,1]):.1f}\n")

print("NPJ: RANGE")
print(f"N_b:   {np.max(w_diff_npj[:,0]) - np.min(w_diff_npj[:,0]):.1f}")
print(f"G_cut: {np.max(w_diff_npj[:,1]) - np.min(w_diff_npj[:,1]):.1f}")

Check how the k-point grid convergence depends on the W parameters.

In [ ]:
# for the k-point convergence we analyze the difference in the gaps
# on different k-point meshes calculated with starting and reference parameters in W
# we check at which k-point grid the gap is converged (difference < 25 meV)
conv_thr = 50  # meV (we define this again to be save)
mat_k_conv = []
counter = -1
for i in range(len(res_cs)):
    min_gaps = np.zeros(5)
    for j in range(5):
        min_gaps[j] = res_cs[i].gwc[j].grid[0, 2]
    ref_gaps = res_ref[i].gap
    gap_diff_min = np.abs(np.diff(min_gaps)) * 1000  # meV
    gap_diff_max = np.abs(np.diff(ref_gaps)) * 1000  # meV
    bool_min = gap_diff_min <= conv_thr
    bool_max = gap_diff_max <= conv_thr
    if any([bool_min[-1] and bool_max[-1]]):
        mat_k_conv.append(np.argmax(bool_min) - np.argmax(bool_max))
        counter += 1
    # in this case we assume that the convergence happens on the next grid
    # this is likely for GaAs (we check the difference in the gaps)
    elif any([bool_min[-1] or bool_max[-1]]):
        print("\nOne did not converge!: ", bool_min[-1], bool_max[-1], res_cs[i].name)
        if not bool_min[-1]:
            temp_min = 3
            temp_max = np.argmax(bool_max)
        if not bool_max[-1]:
            temp_min = np.argmax(bool_min)
            temp_max = 4
        mat_k_conv.append(temp_min - temp_max)
        counter += 1
    else:
        print("\nBoth did not converge: ", bool_min[-1], bool_max[-1], res_cs[i].name)
    if mat_k_conv:
        if mat_k_conv[counter] != 0:
            print("\nDifferent convergence points:", res_cs[i].name)
            print(gap_diff_min)
            print(gap_diff_max)
mat_k_conv = np.array(mat_k_conv)

In [ ]:
# check if the k-point grid convergence is the same for minimal and maximal parameters in W
print(
    f"Number of times where it did not converge at the same point: {len(mat_k_conv[mat_k_conv > 0]) + len(mat_k_conv[mat_k_conv < 0]):d}"
)
print(
    f"Number of times where the convergence happend to late with minimal W parameters: {len(mat_k_conv[mat_k_conv > 0]):d}"
)
print(
    f"Number of times where the convergence happend to early with minimal W parameters: {len(mat_k_conv[mat_k_conv < 0]):d}\n"
)

# same number as percentages
print(
    f"Number of times where it did not converge at the same point: {(len(mat_k_conv[mat_k_conv > 0]) + len(mat_k_conv[mat_k_conv < 0])) / len(mat_k_conv) * 100:.2f} %"
)
print(
    f"Number of times where the convergence happend to late with minimal W parameters: {len(mat_k_conv[mat_k_conv > 0]) / len(mat_k_conv) * 100:.2f} %"
)
print(
    f"Number of times where the convergence happend to early with minimal W parameters: {len(mat_k_conv[mat_k_conv < 0])/ len(mat_k_conv) * 100:.2f} %"
)

In [ ]:
# we investigate the average speedup by converging the
# W parameters on a gamma-only calculation compared to the 2x2x2 k-point grid
# proposed by van Setten et al.
w_speed_up = []
for r in res_cs:
    w_speed_up.append(np.sum(r.gwc[1].grid_time) / np.sum(r.gwc[0].grid_time))
print(
    f"Average speed up for the W parameter convergence on a gamma-only grid: {np.mean(w_speed_up):.1f}"
)
print(
    f"Median speed up for the W parameter convergence on a gamma-only grid: {np.median(w_speed_up):.1f}"
)

In [ ]:
# we investigate the average speedup by converging the
# k-grid with low W parameters compared with the W parameters from the CS convergence
# we note that this is just a approximation as the W parameters at the last point of
# the CS convergence algorithm may change with the k-point grid, however by looking
# at C_p we know that this happens in about 10 % of the cases
# however, by looking at C_P for different k-point grids this effect should average out,
# since the distribution of C_P is roughly symmetric around zero.
k_speed_up = []
for r in res_cs:
    temp_t_min = 0
    temp_t_cs = 0
    for g in r.gwc:
        temp_t_min += g.grid_time[0]
        temp_t_cs += g.grid_time[-1]
    k_speed_up.append(temp_t_cs / temp_t_min)
print(
    f"Average speed up for k-grid convergence with low W parameters: {np.mean(k_speed_up):.1f}"
)
print(
    f"Median speed up for k-grid convergence with low W parameters: {np.median(k_speed_up):.1f}"
)

Plot the data

In [ ]:
# setup publication plot size
width = 3
height = 8

# text size settings
tick_size = int(6 * 1.75)
label_size = int(6 * 1.75)

In [ ]:
# create a plot for the difference between the optimal gap found by the convergence
# algorithms and the reference calculations, which were performed with very high computation parameters
# 2d materials
fig, axs = plt.subplots(5, 2, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(10):
    idx_used_cs = res_cs[i].gap > 0
    idx_used_npj = res_npj[i].gap > 0
    axs[i].plot(
        np.where(idx_used_cs)[0],
        1000 * np.abs(res_cs[i].gap[idx_used_cs] - res_ref[i].gap[idx_used_cs]),
        "ks--",
        markersize=4,
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        1000 * np.abs(res_npj[i].gap[idx_used_npj] - res_ref[i].gap[idx_used_npj]),
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].axhline(y=conv_thr, color="b", linestyle="-", zorder=-1)
    ylimit = axs[i].get_ylim()
    axs[i].set_ylim([0, ylimit[1] * 1.05])
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_\textrm{R}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (meV)",
    x=0.05,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/2d/gw_conv_gap_diff_ref.pdf")

In [ ]:
# create a plot for the number of gw calculation per convergence calculation
# 2d materials
fig, axs = plt.subplots(5, 2, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(10):
    idx_used_cs = res_cs[i].ngw > 0
    idx_used_npj = res_npj[i].ngw > 0
    axs[i].plot(
        np.where(idx_used_cs)[0], res_cs[i].ngw[idx_used_cs], "ks--", markersize=4
    )
    axs[i].plot(
        np.where(idx_used_npj)[0], res_npj[i].ngw[idx_used_npj], "ro--", markersize=4
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_ticks(axs[i].get_yticks().astype(int))
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{GW}$", x=0.05, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/2d/gw_conv_ngw.pdf")

In [ ]:
# create a plot for the computation time for each convergence calculations
# 2d materials
fig, axs = plt.subplots(5, 2, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(10):
    idx_used_cs = res_cs[i].tcpu > 1
    idx_used_npj = res_npj[i].tcpu > 1
    axs[i].plot(
        np.where(idx_used_cs)[0],
        res_cs[i].tcpu[idx_used_cs] / 3600,
        "ks--",
        markersize=4,
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].tcpu[idx_used_npj] / 3600,
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].set_yscale("log")
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[i].yaxis.set_minor_locator(y_minor)
    axs[i].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[i].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{T}_\textrm{CPU}$ (h)", x=0.05, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/2d/gw_conv_tcpu.pdf")

In [ ]:
# create a plot for the convergence parameter N_b for each convergence calculation
# 2d materials
fig, axs = plt.subplots(5, 2, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(10):
    idx_used_cs = res_cs[i].params[:, 0] > 0
    idx_used_npj = res_npj[i].params[:, 0] > 0
    axs[i].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 0], "ks--", markersize=4
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 0],
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 0], res_npj[i].params[idx_used_npj, 0]
                )
            )
        )
    )
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$\textrm{N}_\textrm{b}$", x=0.05, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/2d/gw_conv_nbx.pdf")

In [ ]:
# create a plot for the convergence parameter G_cut for each convergence calculation
# 2d materials
fig, axs = plt.subplots(5, 2, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(10):
    idx_used_cs = res_cs[i].params[:, 2] > 0
    idx_used_npj = res_npj[i].params[:, 2] > 0
    axs[i].plot(
        np.where(idx_used_cs)[0], res_cs[i].params[idx_used_cs, 2], "ks--", markersize=4
    )
    axs[i].plot(
        np.where(idx_used_npj)[0],
        res_npj[i].params[idx_used_npj, 2],
        "ro--",
        markersize=4,
    )
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa))))
    axs[i].xaxis.set_ticklabels(res_cs[i].kpt_str, rotation=45)
    axs[i].yaxis.set_ticks(
        np.sort(
            np.unique(
                np.append(
                    res_cs[i].params[idx_used_cs, 2], res_npj[i].params[idx_used_npj, 2]
                )
            )
        )
    )
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
fig.supxlabel(r"k-point grid", x=0.55, y=0.020, size=label_size)
fig.supylabel(r"$G_\textrm{cut}$ (Ry)", x=0.05, y=0.55, size=label_size)
fig.tight_layout()
fig.savefig("plots/2d/gw_conv_sc.pdf")

In [ ]:
# plot the gap difference for the k-point convergence to show the independence
# of the k-point mesh with respect to the parameters in W
# 2d materials
fig, axs = plt.subplots(5, 2, figsize=(width, height), facecolor="w", dpi=600)
fig.subplots_adjust(hspace=0.2, wspace=0.0)
axs = axs.ravel()
for i in range(10):
    temp_gaps = np.zeros(5)
    for j in range(5):
        temp_gaps[j] = res_cs[i].gwc[j].grid[0, 2]
    axs[i].plot(list(range(4)), np.abs(np.diff(temp_gaps)), "ks--", markersize=4)
    axs[i].plot(list(range(4)), np.abs(np.diff(res_ref[i].gap)), "ro--", markersize=4)
    axs[i].set_title(f"{res_cs[i].name}")
    axs[i].set_xticks([0, 1, 2, 3])
    axs[i].xaxis.set_ticks(np.array(range(len(res_cs[i].kppa) - 1)))
    axs[i].xaxis.set_ticklabels([2, 3, 4, 5])
    axs[i].xaxis.set_tick_params(labelsize=tick_size)
    axs[i].yaxis.set_tick_params(labelsize=tick_size)
    axs[i].set_yscale("log")
    axs[i].axhline(y=0.050, color="b", linestyle="-", zorder=-1)
    y_minor = mticker.LogLocator(
        base=10.0, subs=np.arange(1.0, 10.0) * 0.1, numticks=10
    )
    y_major = mticker.LogLocator(base=10.0, numticks=5)
    axs[i].yaxis.set_minor_locator(y_minor)
    axs[i].yaxis.set_minor_formatter(mticker.NullFormatter())
    axs[i].yaxis.set_major_locator(y_major)
fig.supxlabel(r"k-point grid index $i$", x=0.55, y=0.020, size=label_size)
fig.supylabel(
    r"$\vert\Delta_{\mathbf{k}_i}\textrm{E}_\textrm{gap}^{{\Gamma}-{\Gamma}}\vert$ (eV)",
    x=0.05,
    y=0.55,
    size=label_size,
)
fig.tight_layout()
fig.savefig("plots/2d/gw_conv_gap_diff_kpt.pdf")